In [ ]:
!pip install transformers
!pip install torch
!pip install scikit-learn
!pip install pandas

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer

In [ ]:
data=pd.read_excel('/content/Emotions_DS.xlsx')

In [ ]:
# Extract the input and target columns
X = data['Utterance'].values
y = data['Emotion'].values

In [ ]:
# Encode emotion labels (if not already in numeric form)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [ ]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Tokenize the data
def tokenize_data(text_list):
    # Convert the NumPy array to a list of strings before passing to tokenizer
    # Ensure each element is a string using list comprehension
    text_list = [str(text) for text in text_list]
    return tokenizer(
        text_list,          # Pass the list of strings
        padding=True,        # Pad to the longest sequence
        truncation=True,     # Truncate sequences longer than the max length
        max_length=128,      # Limit the sequence length to 128 tokens
        return_tensors='pt'  # Return PyTorch tensors
    )

In [ ]:
# Tokenize train and test sets
train_encodings = tokenize_data(X_train)
test_encodings = tokenize_data(X_test)

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Create a PyTorch dataset class
class EmotionDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# Create datasets
train_dataset = EmotionDataset(train_encodings, y_train)
test_dataset = EmotionDataset(test_encodings, y_test)

In [ ]:
from transformers import BertForSequenceClassification

# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Move the model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from transformers import AdamW
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

In [ ]:
# Training hyperparameters
batch_size = 16
learning_rate = 5e-5
epochs = 3

In [ ]:
# Initialize data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Initialize optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

In [ ]:
# Training loop
model.train()  # Set the model to training mode

for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for batch in tqdm(train_loader):
        # Move batch to device (GPU or CPU)
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        # Update weights
        optimizer.step()
        optimizer.zero_grad()

    print(f"Loss: {loss.item()}")

Epoch 1/3


100%|██████████| 263/263 [01:25<00:00,  3.08it/s]


Loss: 0.023642737418413162
Epoch 2/3


100%|██████████| 263/263 [01:26<00:00,  3.02it/s]


Loss: 0.5297600030899048
Epoch 3/3


100%|██████████| 263/263 [01:26<00:00,  3.02it/s]

Loss: 0.23720701038837433


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluation
model.eval()  # Set the model to evaluation mode
predictions, true_labels = [], []

with torch.no_grad():  # Disable gradient calculation for evaluation
    for batch in test_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        predictions.extend(torch.argmax(logits, dim=-1).cpu().numpy())
        true_labels.extend(batch['labels'].cpu().numpy())

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.8295


In [ ]:
target_names = [str(cls) for cls in label_encoder.classes_]
report = classification_report(true_labels, predictions, target_names=target_names)
print(report)

              precision    recall  f1-score   support

          -3       0.25      0.08      0.12        26
          -2       1.00      0.03      0.05        39
          -1       0.51      0.63      0.56       147
           0       0.90      0.94      0.92       827
           1       0.00      0.00      0.00         9
           2       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         1

    accuracy                           0.83      1050
   macro avg       0.38      0.24      0.24      1050
weighted avg       0.82      0.83      0.81      1050



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
